In [19]:
from datetime import *
import numpy as np
import pandas as pd
from datascience import *
import re

**Sources of Data**

- The city_table data was manually cleaned from homelessness contracts requested from each city by previous data science interns.  The PIT count data comes from the Regional Task Force on Homelessness, which produces data on number of people experiencing homelessness each calendar year in each city in San Diego County.  I added a column which specifies whether the contract is related to crisis management, prevention, or diversion, based on its specified program.  I then subsetted the data by city and sent the dataframes to each city manager so that they could confirm or correct by classification.  We defined our classifications as follows:

1. Crisis Management, money used explicitly to help people experiencing homelessness (including emergency shelter, drug treatment, etc.)
2. Diversion, money used to help at-risk folks not enter crisis management or rapidly exit (including flexible funds and programs which help reconnect homeless people to friends and family who could take them in temporarily or permanently)
3. Prevention, money used explicitly to prevent individuals from falling into homelessness (including rental assistance, COVID financial relief, etc.)

**Background**

- We would like to ascertain the following: Which type of expenditure in the area of homelessness (crisis management, diversion, or prevention) is most effective (and to what extent) in reducing total number of people experiencing homelessness.  To answer this, we need to clean our data such that we have a data frame which specifies city, calendar year, total number of homeless people, dollars spent in crisis management, prevention and diversion.

In [117]:
city_table = pd.read_csv('CityDollarsTypes.csv')
PIT_table = pd.read_csv('PITCount.csv')

**Let's preview the datasets with which we will be working**

In [10]:
city_table[25:30]

,Unique.ID,Grantor,Grantee,Program,Year,Date,EndDate,Amount,AmendmentNumber,Funding.Agency,...,Issued,Funding.Type,Years,Average.By.Year,City.Year,Population,Amount.Per.Capita,Amount.Per.PEH,Population.PEH,ExpenditureType
25,1066,City of Chula Vista,SBCS CORPORATION,Hotel/Motel Voucher,2018.0,5/4/2018,NaN,"$8,750.60",NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.03,$47.56,184.0,Crisis Management
26,1068,City of Chula Vista,SBCS CORPORATION,Homeless Services,2018.0,5/4/2018,NaN,"$16,825.00",NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.06,$91.44,184.0,Crisis Management
27,1045,City of Chula Vista,SBCS CORPORATION,Rapid Re-Housing,2018.0,5/17/2018,NaN,"$3,583.00",NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.01,$19.47,184.0,Diversion
28,1084,City of Chula Vista,SBCS CORPORATION,Rapid Re-Housing,2018.0,5/17/2018,NaN,"$2,632.75",NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.01,$14.31,184.0,Diversion
29,1067,City of Chula Vista,INTERFAITH SHELTER NETWORK,Rotational Shelter,2018.0,5/21/2018,NaN,"$4,218.84",NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.02,$22.93,184.0,Crisis Management


In [11]:
PIT_table[25:30]

,City,City Remapped,Combined,Year,Full Date,ES,TH,SH,Total - Sheltered,Total - Unsheltered,...,"Unsheltered - Black, African American, or African",Unsheltered - Native Hawaiian or Pacific Islander,Unsheltered - White,Unsheltered - Mulitple Races,"Sheltered - American Indian, Alaska Native, or Indigenous",Sheltered - Asian or Asian American,"Sheltered - Black, African American, or African",Sheltered - Native Hawaiian or Pacific Islander,Sheltered - White,Sheltered - Multiple Races
25,City of Carlsbad,City of Carlsbad,City of Carlsbad|2015,2015.0,NaN,67.0,0.0,0.0,67.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,City of Chula Vista,City of Chula Vista,City of Chula Vista|2022,2022.0,NaN,41.0,62.0,0.0,103.0,206.0,...,14.0,2.0,81.0,1.0,0.0,2.0,18.0,0.0,69.0,11.0
27,City of Chula Vista,City of Chula Vista,City of Chula Vista|2021,2021.0,NaN,80.0,77.0,0.0,157.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,City of Chula Vista,City of Chula Vista,City of Chula Vista|2020,2020.0,NaN,36.0,65.0,0.0,101.0,212.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,City of Chula Vista,City of Chula Vista,City of Chula Vista|2019,2019.0,NaN,NaN,NaN,NaN,79.0,242.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
PIT_table.columns

Index(['City', 'City Remapped', 'Combined', 'Year', 'Full Date', 'ES', 'TH',
       'SH', 'Total - Sheltered', 'Total - Unsheltered', 'Total PEH',
       'Unsheltered - Chronically Homeless', 'Unsheltered - Veteran',
       'Unsheltered - Female', 'Unsheltered - Families', 'Unsheltered - Youth',
       'Sheltered - Chronically Homeless', 'Sheltered - Veteran',
       'Sheltered - Female', 'Sheltered - Families', 'Sheltered - Youth',
       'Unsheltered - Male', 'Unsheltered - Female.1',
       'Unsheltered - Transgender', 'Unsheltered - Questioning',
       'Unsheltered - American Indian, Alaska Native, or Indigenous',
       'Unshelthered - Asian or Asian American',
       'Unsheltered - Black, African American, or African',
       'Unsheltered - Native Hawaiian or Pacific Islander ',
       'Unsheltered - White', 'Unsheltered - Mulitple Races',
       'Sheltered - American Indian, Alaska Native, or Indigenous',
       'Sheltered - Asian or Asian American',
       'Sheltered - Black, 

In [18]:
city_table.columns

Index(['Unique.ID', 'Grantor', 'Grantee', 'Program', 'Year', 'Date', 'EndDate',
       'Amount', 'AmendmentNumber', 'Funding.Agency', 'Funding.Source',
       'Category', 'Location', 'Issued', 'Funding.Type', 'Years',
       'Average.By.Year', 'City.Year', 'Population', 'Amount.Per.Capita',
       'Amount.Per.PEH', 'Population.PEH', 'ExpenditureType'],
      dtype='object')

**Now, let's remove commas and dollar signs from dollar amount column in the expenditure dataset and convert it to a float**

In [130]:
def remove_commas_and_dollar_signs(value):
    pattern = r'[,$]'
    cleaned_value = re.sub(pattern, '', str(value))
    return cleaned_value
city_table['Amount'] = city_table['Amount'].apply(remove_commas_and_dollar_signs)
city_table['Amount'] = city_table['Amount'].apply(float)
city_table[25:28]

,Unique.ID,Grantor,Grantee,Program,Year,Date,EndDate,Amount,AmendmentNumber,Funding.Agency,...,Issued,Funding.Type,Years,Average.By.Year,City.Year,Population,Amount.Per.Capita,Amount.Per.PEH,Population.PEH,ExpenditureType
25,1066,City of Chula Vista,SBCS CORPORATION,Hotel/Motel Voucher,2018.0,5/4/2018,NaN,8750.6,NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.03,$47.56,184.0,Crisis Management
26,1068,City of Chula Vista,SBCS CORPORATION,Homeless Services,2018.0,5/4/2018,NaN,16825.0,NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.06,$91.44,184.0,Crisis Management
27,1045,City of Chula Vista,SBCS CORPORATION,Rapid Re-Housing,2018.0,5/17/2018,NaN,3583.0,NaN,NaN,...,NaN,NaN,NaN,NaN,City of Chula Vista|2018,268588.0,$0.01,$19.47,184.0,Diversion


**Next, we need to subset the tables based on what columns we want for the analysis**

The point of this is to remove unnecessary columns/rows to simplify our dataset.  We then merge the two subsets to create a dataframe which has all the info we need for our analysis.

In [131]:
city_table_subset = city_table[['Grantor', 'Year', 'Amount', 'ExpenditureType']]
PIT_table_subset = PIT_table[['City Remapped','Year','Total PEH']]
final_table = pd.merge(city_table_subset, PIT_table_subset, left_on=['Grantor', 'Year'], right_on=['City Remapped', 'Year'], how='inner')
final_table = final_table[final_table['Amount'] != 0]
final_table = final_table.dropna(subset=['Amount'])
final_table = final_table.reset_index(drop=True)

**Next, I need to further clean our dataset by converting final_table from a series to a dataframe, then converting column 'Year' to an integer.**

In [132]:
if isinstance(final_table, pd.Series):
    final_table = final_table.to_frame()

In [133]:
final_table['Year'] = final_table['Year'].astype(int)
final_table

,Grantor,Year,Amount,ExpenditureType,City Remapped,Total PEH
0,City of Chula Vista,2017,14142.74,Prevention,City of Chula Vista,367
1,City of Chula Vista,2017,12931.10,Prevention,City of Chula Vista,367
2,City of Chula Vista,2017,10285.61,Prevention,City of Chula Vista,367
3,City of Chula Vista,2017,276.00,Crisis Management,City of Chula Vista,367
4,City of Chula Vista,2017,1971.50,Crisis Management,City of Chula Vista,367
...,...,...,...,...,...,...
723,City of Oceanside,2021,96487.59,Prevention,City of Oceanside,145
724,City of Oceanside,2021,114050.00,Crisis Management,City of Oceanside,145
725,City of Oceanside,2021,767500.00,Crisis Management,City of Oceanside,145
726,City of Oceanside,2021,93452.38,Prevention,City of Oceanside,145


In [134]:
final_table = final_table.drop('City Remapped', axis=1)
final_table[400:410]

,Grantor,Year,Amount,ExpenditureType,Total PEH
400,City of Chula Vista,2021,1094.00,Prevention,157
401,City of Chula Vista,2021,3205.34,Crisis Management,157
402,City of Chula Vista,2021,4989.36,Crisis Management,157
403,City of Chula Vista,2021,12217.00,Prevention,157
404,City of Chula Vista,2021,7891.74,Crisis Management,157
405,City of Chula Vista,2021,1948.00,Diversion,157
406,City of Chula Vista,2021,2959.00,Diversion,157
407,City of Chula Vista,2021,3393.00,Diversion,157
408,City of Chula Vista,2021,1094.00,Prevention,157
409,City of Chula Vista,2021,553.00,Crisis Management,157


In [135]:
final_table = final_table[final_table['ExpenditureType'] != "Other/Unknown"]
final_table = final_table.reset_index(drop=True)
final_table

,Grantor,Year,Amount,ExpenditureType,Total PEH
0,City of Chula Vista,2017,14142.74,Prevention,367
1,City of Chula Vista,2017,12931.10,Prevention,367
2,City of Chula Vista,2017,10285.61,Prevention,367
3,City of Chula Vista,2017,276.00,Crisis Management,367
4,City of Chula Vista,2017,1971.50,Crisis Management,367
...,...,...,...,...,...
721,City of Oceanside,2021,96487.59,Prevention,145
722,City of Oceanside,2021,114050.00,Crisis Management,145
723,City of Oceanside,2021,767500.00,Crisis Management,145
724,City of Oceanside,2021,93452.38,Prevention,145


**Next, we need to make a new dataframe which aggregates Crisis Management, Diversion, and Prevention Dollars by city and year**

In [136]:
grouped_table = final_table.groupby(['Year', 'Grantor', 'ExpenditureType'])['Amount'].sum().reset_index()
pivot_table = grouped_table.pivot_table(index=['Year', 'Grantor'], columns='ExpenditureType', values='Amount', aggfunc='sum').reset_index()
pivot_table.columns.name = None
pivot_table = pivot_table.rename(columns={'Crisis Management': 'CM_Amount', 'Diversion': 'D_Amount', 'Prevention': 'P_Amount'})
final_table = pd.merge(pivot_table, final_table[['Year', 'Grantor', 'Total PEH']].drop_duplicates(), on=['Year', 'Grantor'])
final_table = final_table[['Year', 'Grantor', 'Total PEH', 'CM_Amount', 'D_Amount', 'P_Amount']]
final_table

,Year,Grantor,Total PEH,CM_Amount,D_Amount,P_Amount
0,2015,City of El Cajon,711,240000.00,NaN,NaN
1,2015,City of San Marcos,0,17408.00,NaN,21600.0
2,2016,City of El Cajon,321,231854.00,NaN,NaN
3,2016,City of San Marcos,99,23210.67,NaN,27600.0
4,2017,City of Carlsbad,160,14896.00,NaN,NaN
...,...,...,...,...,...,...
62,2022,City of Del Mar,0,5560.00,NaN,NaN
63,2022,City of El Cajon,1308,310000.00,NaN,NaN
64,2022,City of Encinitas,113,162500.00,NaN,NaN
65,2022,City of La Mesa,53,574000.00,NaN,NaN


**Create a new column which calculates the total amount spent for each city each year**

In [137]:
final_table['CM_Amount'].fillna(0, inplace=True)
final_table['D_Amount'].fillna(0, inplace=True)
final_table['P_Amount'].fillna(0, inplace=True)
final_table['Total Amount'] = final_table['CM_Amount'] + final_table['D_Amount'] + final_table['P_Amount']
final_table

,Year,Grantor,Total PEH,CM_Amount,D_Amount,P_Amount,Total Amount
0,2015,City of El Cajon,711,240000.00,0.0,0.0,240000.00
1,2015,City of San Marcos,0,17408.00,0.0,21600.0,39008.00
2,2016,City of El Cajon,321,231854.00,0.0,0.0,231854.00
3,2016,City of San Marcos,99,23210.67,0.0,27600.0,50810.67
4,2017,City of Carlsbad,160,14896.00,0.0,0.0,14896.00
...,...,...,...,...,...,...,...
62,2022,City of Del Mar,0,5560.00,0.0,0.0,5560.00
63,2022,City of El Cajon,1308,310000.00,0.0,0.0,310000.00
64,2022,City of Encinitas,113,162500.00,0.0,0.0,162500.00
65,2022,City of La Mesa,53,574000.00,0.0,0.0,574000.00


**Since the Point-in-Time data varies greatly, based on city population, we need to gather population data in order to compute per capita**

Fortunately, we already have this data.  It needs to be cleaned and added to our final dataframe.

In [138]:
city_table_pop_subset = city_table[['Population','Grantor','Year']]
city_table_pop_subset[25:30]
popdf = pd.DataFrame(city_table_pop_subset)
popdf = popdf.drop_duplicates()
popdf = popdf.dropna(subset=['Population'])
popdf = popdf.reset_index(drop=True)
popdf['Year'] = popdf['Year'].astype(int)
final_table = pd.merge(popdf, final_table, on=['Year', 'Grantor'], how='inner')
final_table

,Population,Grantor,Year,Total PEH,CM_Amount,D_Amount,P_Amount,Total Amount
0,266427.0,City of Chula Vista,2017,367,8166.70,0.00,118791.52,126958.22
1,268588.0,City of Chula Vista,2018,184,151495.98,217926.00,208311.42,577733.40
2,271362.0,City of Chula Vista,2019,321,230954.12,184040.78,429976.02,844970.92
3,273384.0,City of Chula Vista,2020,313,152864.70,105212.00,299920.22,557996.92
4,277220.0,City of Chula Vista,2021,157,440129.00,52986.00,63728.00,556843.00
...,...,...,...,...,...,...,...,...
60,61762.0,City of Encinitas,2021,48,825561.50,0.00,24512.00,850073.50
61,62394.0,City of Encinitas,2018,60,153319.00,0.00,13407.00,166726.00
62,62296.0,City of Encinitas,2019,120,197120.00,0.00,13607.00,210727.00
63,61762.0,City of Encinitas,2022,113,162500.00,0.00,0.00,162500.00


**Now, let's calculate PEH per capita**

In [139]:
final_table['PEH Per 10,000'] = (final_table['Total PEH']/final_table['Population']) * 10000
final_table

,Population,Grantor,Year,Total PEH,CM_Amount,D_Amount,P_Amount,Total Amount,"PEH Per 10,000"
0,266427.0,City of Chula Vista,2017,367,8166.70,0.00,118791.52,126958.22,13.774880
1,268588.0,City of Chula Vista,2018,184,151495.98,217926.00,208311.42,577733.40,6.850641
2,271362.0,City of Chula Vista,2019,321,230954.12,184040.78,429976.02,844970.92,11.829217
3,273384.0,City of Chula Vista,2020,313,152864.70,105212.00,299920.22,557996.92,11.449097
4,277220.0,City of Chula Vista,2021,157,440129.00,52986.00,63728.00,556843.00,5.663372
...,...,...,...,...,...,...,...,...,...
60,61762.0,City of Encinitas,2021,48,825561.50,0.00,24512.00,850073.50,7.771769
61,62394.0,City of Encinitas,2018,60,153319.00,0.00,13407.00,166726.00,9.616309
62,62296.0,City of Encinitas,2019,120,197120.00,0.00,13607.00,210727.00,19.262874
63,61762.0,City of Encinitas,2022,113,162500.00,0.00,0.00,162500.00,18.296040


**Since our data is now cleaned, we can perform some prelimenary regression analysis**

In the first cell, we are computing the observed effect that each type of expenditure has on the number of homeless people per 10,000.  In the second cell, we are computing the effect spending in general has on the number of homeless people per 10,000. 

In [149]:
from sklearn.linear_model import LinearRegression
X = final_table[['CM_Amount','D_Amount','P_Amount']]
y = final_table['PEH Per 10,000']
reg1 = LinearRegression().fit(X, y)
print('Coefficients:', reg1.coef_)
print('Intercept:', reg1.intercept_)
print('R-squared:', reg1.score(X, y))
print('R:', (reg1.score(X, y))**0.5)

Coefficients: [  8.82724580e-07   1.82591918e-05  -1.75057069e-05]
Intercept: 20.1790783631
R-squared: 0.0262411893943
R: 0.161991325059


In [150]:
X2 = final_table[['Total Amount']]
y2 = final_table['PEH Per 10,000']
reg2 = LinearRegression().fit(X2, y2)
print('Coefficients:', reg2.coef_)
print('Intercept:', reg2.intercept_)
print('R-squared:', reg2.score(X2, y2))
print('R:', (reg2.score(X2, y2))**0.5)

Coefficients: [  1.78660982e-07]
Intercept: 19.1971975828
R-squared: 0.000889223342013
R: 0.0298198481219


**Results:**

The most important thing to note here is that there is very little correlation observed between spending and number of people experiencing homelessness.  Less than three percent of the variation in people experiencing homelessness is caused by spending in the three areas of homelessness.  What this tells us is that the money going towards these programs is not being utilized well.

- All else held constant, for every additional 10 million dollars spent in Crisis Management programs, 8.8272458 **ADDITIONAL** people per 10,000 are expected to experience homelessness.
- All else held constant, for every additional 10 million dollars spent in Diversion programs, 182.591918 **ADDITIONAL** people per 10,000 are expected to experience homelessness.
- All else held constant, for every additional 10 million dollars spent in Prevention programs, 175.057069 **FEWER** people per 10,000 are expected to experience homelessness.

Our results indicate that prevention is by far the most effective area of spending.  This is consistent with data from RTFH which has said that the ratio of people exiting homelessness to people entering homelessness decreased from 10:13 to 10:16 between 2022 and 2023.